In [32]:
#버전1
from urllib.parse import urlencode, quote_plus
import requests
import pandas as pd
from bs4 import BeautifulSoup
import bs4

url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?_wadl&type=xml'
service_key = 'OGhTjetKP%2Bhch%2BVsdkrc25WzVEpvhA0HiAwPfw%2Fx1ApwnvAEkGDxZGmq8Tnq2m0q1I%2BdC6TH9gdzuwHiVAUVDg%3D%3D'

base_date = ["202007", "202008", "202009"]

for i in range(len(base_date)):
    
    #구 단위로 데이터 수집 ex) 금천구
    gu_code = '11545'
    payload = "serviceKey = " + service_key + "LAWD_CD = " + gu_code + "&" + "DEAL_YMD = " + base_date[i] + "&"
    
    res = requests.get(url + payload).text
    xmlobj = bs4.BeautifulSoup(res, 'lxml-xml')
    rows = xmlobj.findAll('item')
    
    rowList = []
    nameList = []
    columnList = []
    
    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()
        
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name)
            eachColumn = columns[j].text
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []
    result = pd.DataFrame(rowList, columns = nameList)

In [33]:
result

""


In [29]:
#버전2
import os
import pandas as pd
import urllib.request
from requests import get
from bs4 import BeautifulSoup

serviceKey = 'OGhTjetKP%2Bhch%2BVsdkrc25WzVEpvhA0HiAwPfw%2Fx1ApwnvAEkGDxZGmq8Tnq2m0q1I%2BdC6TH9gdzuwHiVAUVDg%3D%3D'
ymd = '201512'
landcode = '11110'
numrow = 1000
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?LAWD_CD={11110}&DEAL_YMD={201512}&serviceKey={OGhTjetKP%2Bhch%2BVsdkrc25WzVEpvhA0HiAwPfw%2Fx1ApwnvAEkGDxZGmq8Tnq2m0q1I%2BdC6TH9gdzuwHiVAUVDg%3D%3D&numOfRows={1000}'

def exportExcel2(itemList, fileName):
    
    writer = pd.ExcelWriter(fileName, engine='xlsxwriter')
    df = pd.DataFrame(columns=itemList[0])
    
    for data in itemList:
        item = data
        
        df.loc[len(df)] = data
        
        print(item)
        
        df = df.sort_values(by='매매일', ascending=False)
        df.to_excel(writer, sheet_name='마포구')
        writer.save()
        os.startfile(fileName)
        return
    
print(url)

res = urllib.request.urlopen(url)
result = res.read()
soup = BeautifulSoup(result, 'lxml-xml')
items = soup.findAll('item')

keylist = {
    '거래금액',
    '건축년도',
    '년',
    '도로명',
    '도로명건물본번호코드',
    '도로명건물부번호코드',
    '도로명시군구코드',
    '도로명일련번호코드',
    '도로명코드',
    '법정동',
    '법정동본번코드',
    '법정동부번코드',
    '법정동시군구코드',
    '법정동읍면동코드',
    '법정동지번코드',
    '아파트',
    '월',
    '일',
    '전용면적',
    '지번',
    '지역코드',
    '층',
}

itemList = []

for v in items:
    item = {}
    
    for key in keylist:
        item[key] = v.find(key).text
    
    item['매매일'] = int(item['년']) * 10000 + int(item['월']) * 100 + int(item['일'])
    item['거래금액'] = int(item['거래금액'].replace(',', ''))
    item['전용면적'] = float(item['전용면적'])
    item['층'] = int(item['층'])
    item['건축년도'] = int(item['건축년도'])
    itemList.append(item)
    
exportExcel2(itemList, 'C:\\Users\\USER\\landlist.xlsx')

http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?LAWD_CD={11110}&DEAL_YMD={201512}&serviceKey={OGhTjetKP%2Bhch%2BVsdkrc25WzVEpvhA0HiAwPfw%2Fx1ApwnvAEkGDxZGmq8Tnq2m0q1I%2BdC6TH9gdzuwHiVAUVDg%3D%3D&numOfRows={1000}


ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
#버전3
import pandas as pd
import requests
import bs4 as bs
import urllib.request

from general_functions import get_file_contents
from general_functions import load_xml_to_dataframe


url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
filename = 'ServiceKey 국토교통부 아파트매매 실거래.txt'
api_key = get_file_contents(filename)

In [ ]:
df = pd.DataFrame()
for ymd in range(202101, 202205):  # ['202101', '202204', '202205']:
    params = {'serviceKey': api_key, 'LAWD_CD': '50130', 'DEAL_YMD': str(ymd)}
    try:
        d = load_xml_to_dataframe(requests.get(url,
                                           params=params
                                           ).text, 'item')
        df = df.append(d, ignore_index=True)
    except:
        pass

In [ ]:
df['거래일자'] = df.년.astype(str) + '-' + df.월.astype(str) + '-' + df.일.astype(str)
df['거래일자'] = pd.to_datetime(df['거래일자'])

In [ ]:
for col in ['거래금액', '건축년도', '년', '층', '일', '월']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""))

In [ ]:
import numpy as np
pd.pivot_table(data=df, index='법정동', columns='거래유형',
               values='거래금액', aggfunc=sum, fill_value=0)

In [ ]:
import pydeck as pdk
import streamlit as st

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
token = os.getenv('MAPBOX_API_KEY')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

sns.set_palette('Set2')

plt.rcParams['font.family'] = 'AppleGothic'

#figsize 를 전역으로 설정
plt.rcParams['figure.figsize'] = (13, 8)

In [ ]:
result = pd.pivot_table(data=df, index='법정동', values='거래금액',
                        aggfunc='sum', fill_value=0).sort_values('거래금액', ascending=False)
result[:10]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
import plotly.express as px

fig = px.scatter(
    df,
    color='법정동',
    x=df.거래일자.sort_values(ascending=True),
    y=df.거래금액
    )

fig.show()

fig.write_html("output.html")

In [ ]:
df.info()

In [ ]:
import datetime

fig = px.bar(
    df,
    color='법정동',
    x= df['거래일자'].dt.strftime('%Y-%m') ,
    y='거래금액'
    )

fig.show()